In [ ]:
if "google.colab" in str(get_ipython()):
    !git clone https://github.com/lukebella/YelpRecommenderSystem.git
    !mv YelpRecommenderSystem/* .
    !rm -fr YelpRecommenderSystem


In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = "xxxxxx"
os.environ['KAGGLE_KEY'] = "xxxxxx"
!kaggle datasets download -p ./data -d yelp-dataset/yelp-dataset
!unzip -n ./data/yelp-dataset.zip -d ./data


In [ ]:
review_filename = 'data/yelp_academic_dataset_review.json'
user_filename = 'data/yelp_academic_dataset_user.json'

In [ ]:
import pyspark
from pyspark.sql import SparkSession

# .master().config('spark.driver.memory', "15g")
# sc = pyspark.SparkContext().getOrCreate()
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext


In [ ]:
# TODO add docstring
def from_json_to_RDD(filename):
    raw_df = spark.read.json(filename)
    # raw_df.printSchema()
    raw_df.show()
    return raw_df.rdd

raw_review_RDD = from_json_to_RDD(review_filename)
raw_user_RDD = from_json_to_RDD(user_filename)

In [ ]:
def get_review_tuple(entry):
    """ Parse a line in the review dataset form pyspark.sql.Row to tuple
    Args:
        entry (pyspark.sql.Row): a line in the review dataset in JSON format
    Returns:
        tuple: (review_id, user_id, business_id, stars, useful, funny, cool, text)
    """

    return (str(entry["review_id"]),    # 0
            str(entry["user_id"]),      # 1
            str(entry["business_id"]),  # 2
            int(entry["stars"]),        # 3
            int(entry["useful"]),       # 4
            int(entry["funny"]),        # 5
            int(entry["cool"]),         # 6
            str(entry["text"]))         # 7


review_RDD = raw_review_RDD.map(get_review_tuple)

review_count = review_RDD.count()

print(f'There are {review_count} reviews in the dataset')
print(f'Reviews: {review_RDD.take(3)}') # TODO fix print


In [ ]:
# TODO fix docstring
def get_user_tuple(entry):
    """ Parse a line in the review dataset form pyspark.sql.Row to tuple
    Args:
        entry (pyspark.sql.Row): a line in the review dataset in JSON format
    Returns:
        tuple: (review_id, user_id, business_id, stars, useful, funny, cool, text)
    """

    return (str(entry["user_id"]),          # 0
            str(entry["name"]),             # 1
            int(entry["review_count"]),     # 2
            float(entry["average_stars"]),  # 3
            int(entry["useful"]),           # 4
            int(entry["funny"]),            # 5
            int(entry["cool"]),             # 6
            int(entry["fans"]))             # 7


user_RDD = raw_user_RDD.map(get_user_tuple)

user_count = user_RDD.count()

print(f'There are {user_count} users in the dataset')
print(f'Users: {user_RDD.take(3)}') # TODO fix print


In [ ]:
#review_RDD.filter(lambda x: x[3]==0).count()


#stars - useful - funny - cool - numero fan


#if stars <3
    #stars - {[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2] + (numero fan/best user fans)}/2
#else
    #stars + {[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2] + (numero fan/best user fans)}/2

    #dati mancanti: numero fan; best fan; best useful/funny/cool per ristorante
#rdd_test_ufc = (id ristorsante, best useful, best funny, best cool)
#best fan = query su dataset utenti --> user_RDD.max().first()

#review_id - user_id - id_rist - overall


        


In [ ]:
# review_best_ufc_RDD tuple: (business_id, (useful, funny, cool)): for each reastaurant, this tuple takes the maxima values of useful, funny and cool. 
review_best_ufc_RDD = review_RDD.map(lambda x : (x[2], (x[4], x[5], x[6]))).reduceByKey(lambda x, y : tuple(max(x[i], y[i]) for i in range(len(y))))

review_best_ufc_RDD.take(5)

In [ ]:
best_user_fan = user_RDD.max(lambda x:x[7])
print(best_user_fan)

In [ ]:
#get_overall_rating: function that generates the bias in order to incentivate or decrease the importance or overall rating of a review

def get_overall_rating(stars, best_useful, best_funny, best_cool, useful, funny, cool, fans):
    overall = ((useful + (funny + cool)/2) / (best_useful + (best_funny + best_cool)/2) + (fans/best_user_fan)) /2
    if stars < 3:
        return stars - overall
    else: 
        return stars + overall


In [ ]:
user_fan_RDD = user_RDD.map(lambda x: (x[0],x[7]))
review_fans_RDD = review_RDD.map(lambda x: (x[1], x[0], x[2], x[3], x[4], x[5], x[6])).join(user_fan_RDD)
review_fans_RDD.take(3)

In [ ]:

user_fan_RDD.join(review_RDD.map(lambda x: (x[1], (x[0], x[2], x[3], x[4], x[5], x[6])))).first()